In [14]:
from serpapi import GoogleSearch
import json
import os
from dotenv import load_dotenv
from anthropic import Anthropic


load_dotenv()

True

In [ ]:

CLAUDE_API_KEY = os.environ.get("CLAUDE_API_KEY")
client = Anthropic(api_key=CLAUDE_API_KEY)

system_prompt = ""

try:
    with open("hotels_prompt1.txt", "r", encoding="utf-8") as f:
        system_prompt = f.read()
        
except FileNotFoundError:
    print("hotels_prompt1.txt not found")
    
else:
    def get_hotel_request(user_input):
        response = client.messages.create(
            model="claude-sonnet-4-20250514",   
            max_tokens=500,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_input}
            ]
        )
        raw = response.content[0].text.strip()
        return raw

    user_text = "Book me a hotel in Dubai for 2 adults and 1 kid aged 6 from Jan 10 to Jan 15"
    hotelRequest = get_hotel_request(user_text)
    print(hotelRequest)


params = {
  "engine": "google_hotels",
  "q": "Hotels in Dubai",
  "check_in_date": "2025-01-10",
  "check_out_date": "2025-01-15",
  "adults": "2",
  "children": "1",
  "children_ages": "6",
  "currency": "SAR",
  "gl": "sa",
  "hl": "en",
  "api_key": ""
}


In [15]:
SERPAPI_KEY = os.environ.get("SERPAPI_KEY")
params = {
    "engine": "google_hotels",
    "q": "Riyadh hotels",
    "check_in_date": "2025-11-13",
    "check_out_date": "2025-11-14",
    "adults": "2",
    "sort_by": "8",
    "max_price": "500",
    "currency": "SAR",
    "gl": "SA",
    "hl": "en",
    "api_key": SERPAPI_KEY
}

search = GoogleSearch(params)
results = search.get_dict()

with open("hotels.json", "w") as f:
    json.dump(results, f, indent=2)

In [16]:
clean_hotels = []
for hotel in results.get("properties", []):
    clean_hotels.append({
        "id": hotel.get("property_token"),
        "name": hotel.get("name"),
        "price": hotel.get("rate_per_night", {}).get("extracted_lowest"),
        "stars": hotel.get("extracted_hotel_class"),
        "rating": hotel.get("overall_rating"),
        "reviews": hotel.get("reviews"),
        "location_rating": hotel.get("location_rating"),
        "coordinates": hotel.get("gps_coordinates"),
        "amenities": hotel.get("amenities", [])[:5],
        "link": hotel.get("link"),
        "sponsored": hotel.get("sponsored")
    })


out_file = "hotels_filtered.json"
with open(out_file, "w", encoding="utf-8") as f:
    json.dump(clean_hotels, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(clean_hotels)} hotels to {out_file}")


Wrote 18 hotels to hotels_filtered.json
